In [1]:
#from models import keras_1, keras_2
from sklearn.model_selection import train_test_split
from preprocess import *
from evaluation import *
from common import *
# train a feats model
from gen_feats_data import get_feats_data
from gen_fieldpos_data import get_fieldpos_data
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
from sklearn.feature_selection import f_classif, SelectKBest , chi2, mutual_info_classif,SelectPercentile

def feature_selection_evaluation(tr_data,val_data,k_values, keras_model):
    accuracies = []
    
    for kval in k_values:
        
        kbest = SelectKBest( score_func = mutual_info_classif,k = kval)
        
        feat_sel_tr_data = kbest.fit_transform(tr_data[0],onehot(tr_data[1]))
        
        feat_sel_val_data = kbest.transform(val_data[0])

        #get all the selected features and apply them to the validation data 

        model = keras_model(feat_sel_tr_data,onehot(tr_data[1]),(feat_sel_val_data,onehot(val_data[1])))
        val_accuracy =  model.evaluate(feat_sel_val_data,onehot(val_data[1]))[1] #only accuracy
        accuracies.append(val_accuracy)
        print(kval, val_accuracy)
        # Get idxs of columns to keep
        cols = kbest.get_support(indices=True)
        # Create new dataframe with only desired columns, or overwrite existing
        sel_cols = [tr_data[0].columns[ic] for ic in cols]
        print(sel_cols)

    return accuracies

In [41]:
from sklearn.feature_selection import f_classif, SelectKBest , chi2, mutual_info_classif,SelectPercentile

def feature_selection_pca(tr_data,val_data,k_values, keras_model):
    accuracies = []
    
    for kval in k_values:
        
        kbest = SelectKBest( k = kval)
        
        feat_sel_tr_data = kbest.fit_transform(tr_data[0],tr_data[1])
        
        feat_sel_val_data = kbest.transform(val_data[0])

        #get all the selected features and apply them to the validation data 

        model = keras_model(feat_sel_tr_data,onehot(tr_data[1]),(feat_sel_val_data,onehot(val_data[1])))
        val_accuracy =  model.evaluate(feat_sel_val_data,onehot(val_data[1]))[1] #only accuracy
        accuracies.append(val_accuracy)
        print(kval, val_accuracy)
        # Get idxs of columns to keep
        cols = kbest.get_support()
        # Create new dataframe with only desired columns, or overwrite existing
        print(cols)
    return accuracies

In [5]:
feats_tr_data = get_feats_data(['2014','2015','2016'])
feats_val_data = get_feats_data(['2017'])

In [ ]:
print(feats_tr_data[0].columns)
x = feats_tr_data[0]['finishing']
y = feats_tr_data[1]
#sns.jointplot(feats_tr_data[0]['sprint_speed'],feats_tr_data[1], kind = 'kde')

f, ax = plt.subplots(figsize=(6, 6))
cmap = sns.cubehelix_palette(as_cmap=True, dark=1, light=0, reverse=True)
sns.kdeplot(x,y , cmap=cmap, n_levels=100, shade=True);

with sns.axes_style("white"):
    sns.jointplot(x=x, y=y, kind="hex", color="k");

In [ ]:
print(feats_val_data)

In [47]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import regularizers
import numpy as np

# class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
# sample_weights = compute_sample_weight('balanced', y_integers)

def keras_1(x_train,y_train,val_tuple):
#     from sklearn.utils import compute_class_weight
#     classWeight = compute_class_weight('balanced', np.unique(y_train), y_train)
#     classWeight = dict(enumerate(classWeight,1))
#     print(classWeight)
    
    from keras.optimizers import SGD
    opt = SGD(lr=0.05)
    """ Creates a Sequential model made out of Densely connected Neural Network layers. All parameters are configured as per specification for Q4"""

    model = Sequential()
    model.add(Dense(100,input_dim = x_train.shape[1], activation = 'sigmoid'))
#     model.add(Dense(200, activation = 'tanh'))
#     model.add(Dropout(0.1))
#     model.add(Dense(50, activation = 'tanh'))
    model.add(Dense(3, activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer ='adam',metrics = ['accuracy'])

    model.fit(x_train,y_train,epochs = 50, batch_size = 32, validation_data = val_tuple,verbose=0)
    print(model.evaluate(val_tuple[0],val_tuple[1]))
    return model#.predict(val_tuple[0])
   
def keras_2(x_train,y_train,val_tuple):
    """ Creates a Sequential model made out of Densely connected Neural Network layers. All parameters are configured as per specification for Q4"""

    from keras.optimizers import SGD
    opt = SGD(lr=0.05)
    """ Creates a Sequential model made out of Densely connected Neural Network layers. All parameters are configured as per specification for Q4"""

    model = Sequential()
    model.add(Dense(100,input_dim = x_train.shape[1], activation = 'sigmoid'))
    model.add(Dense(200, activation = 'sigmoid'))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer ='adam',metrics = ['categorical_crossentropy'])

    model.fit(x_train,y_train,epochs = 50, batch_size = 32, validation_data = val_tuple)
    #print(model.evaluate())
    #print(model.metrics)
    return model#.predict(val_tuple[0])

In [54]:
#select k best

feature_selection_evaluation(feats_tr_data, feats_val_data, [5,8,14,17,20,27],keras_1)


D:\Apps\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


ValueError: bad input shape (1140, 3)

In [ ]:
model = keras_1(feats_tr_data[0],onehot(feats_tr_data[1]),(feats_val_data[0],onehot(feats_val_data[1])))
# inv_pred = inv_onehot(feats_pred).reshape(1,-1)[0]
# acc = accuracy_score(inv_pred,feats_val_data[1])
# #acc


In [3]:
feats_test_data = get_feats_data(['2018'])
feats_test_data[0].shape

(380, 28)

In [15]:
test_pred = model.predict(feats_test_data[0])
inv_test_pred = inv_onehot(test_pred).reshape(1,-1)[0]
print(test_pred)
print(inv_test_pred)

[[0.62609214 0.23520498 0.13944758]
 [0.14751111 0.24916524 0.67859584]
 [0.5434143  0.23962684 0.1930917 ]
 ...
 [0.5401246  0.24014941 0.1954473 ]
 [0.65614617 0.2330242  0.12251051]
 [0.25752464 0.24854277 0.49283305]]
[1. 3. 1. 3. 3. 1. 1. 1. 1. 3. 3. 1. 1. 1. 1. 1. 3. 1. 3. 3. 3. 1. 3. 1.
 3. 1. 1. 1. 3. 3. 3. 1. 1. 1. 1. 1. 1. 3. 3. 3. 1. 3. 1. 3. 1. 3. 3. 1.
 1. 3. 3. 3. 3. 1. 1. 1. 1. 3. 3. 1. 3. 1. 1. 3. 1. 3. 1. 3. 3. 1. 3. 1.
 3. 1. 1. 1. 1. 3. 3. 1. 3. 1. 3. 1. 3. 3. 1. 3. 1. 1. 3. 1. 1. 1. 1. 1.
 3. 3. 1. 1. 3. 3. 1. 3. 1. 3. 3. 1. 1. 3. 3. 3. 1. 3. 3. 3. 1. 1. 1. 1.
 3. 1. 1. 1. 3. 3. 1. 3. 1. 1. 3. 1. 3. 1. 1. 3. 3. 1. 3. 1. 1. 1. 1. 3.
 3. 3. 3. 1. 1. 1. 3. 1. 3. 3. 1. 1. 3. 3. 3. 1. 1. 3. 3. 1. 1. 1. 3. 3.
 3. 1. 3. 1. 3. 1. 3. 1. 3. 1. 3. 3. 3. 3. 1. 3. 1. 1. 1. 3. 3. 3. 3. 1.
 1. 1. 1. 1. 3. 3. 3. 1. 3. 1. 3. 3. 1. 3. 1. 1. 3. 1. 3. 3. 1. 1. 3. 1.
 1. 1. 1. 1. 1. 3. 1. 1. 1. 3. 1. 3. 3. 3. 1. 3. 3. 3. 1. 1. 3. 3. 1. 1.
 1. 1. 3. 1. 1. 3. 3. 1. 1. 3. 1. 3. 3. 3. 1. 1.

In [ ]:
fieldpos_tr_data = get_fieldpos_data(['2014','2015','2016'])
fieldpos_val_data = get_fieldpos_data(['2017'])

In [ ]:
fieldpos_test_data = get_fieldpos_data(['2018'])

In [ ]:
model_2 = keras_2(fieldpos_tr_data[0],onehot(fieldpos_tr_data[1]),(fieldpos_val_data[0],onehot(fieldpos_val_data[1])))
# print(fieldpos_pred)
# inv_pred = inv_onehot(fieldpos_pred).reshape(1,-1)[0]
# print(inv_pred)

#accuracy_score(inv_pred,fieldpos_val_data[1])
print(model_2)
test_pred_2 = model_2.predict(fieldpos_test_data[0])
inv_test_pred_2 = inv_onehot(test_pred_2).reshape(1,-1)[0]


In [ ]:



it = feats_val_data[0].iterrows()

for i in range(0,len(feats_val_data[1])):
    print(feats_pred[i], fieldpos_pred[i],feats_val_data[1][i],  [feats_pred[i][0] +   fieldpos_pred[i][0],  feats_pred[i][1] + fieldpos_pred[i][1],  feats_pred[i][2] + fieldpos_pred[i][2]])


In [ ]:
import operator

it = feats_val_data[0].iterrows()
z = 0
for i in range(0,len(feats_val_data[1])):
    a = [0,feats_pred[i][0],feats_pred[i][1],feats_pred[i][2],0,fieldpos_pred[i][0], fieldpos_pred[i][1],fieldpos_pred[i][2]]
    m = max(a)
    i = a.index(m)
    if (i)%4 ==feats_val_data[1][i] :
        z = z + 1
z/len(feats_val_data[1])

In [7]:
def generate_final_table(fixtures, results, total_games):
    team_points = {}

    for i in range(10):
        team_points[fixtures.iloc[i]['HomeTeam']] = 0
        team_points[fixtures.iloc[i]['AwayTeam']] = 0

    for i in range(total_games):
        result = results[i]
        fixture = fixtures.iloc[i]
        if(result == 1):
            team_points[fixture['HomeTeam']] += 3
        elif(result == 2):
            team_points[fixture['HomeTeam']] += 1
            team_points[fixture['AwayTeam']] += 1
        elif(result == 3):
            team_points[fixture['AwayTeam']] += 3

    sorted_team_points = sorted(team_points.items(), key=lambda kv: kv[1])
    sorted_team_points.reverse()
    return dict(sorted_team_points)

In [8]:
import pandas as pd

results = []
fixtures = get_fixtures('2018')
#fixtures = fixtures[fixtures['Round Number'] <= 16]
fixtures.shape

(380, 11)

In [10]:

final_table = pd.DataFrame.from_dict(generate_final_table(fixtures, inv_test_pred,380), orient = 'index')
final_table 




(380,)
[1. 3. 1. 1. 3. 1. 1. 1. 1. 3. 3. 1. 1. 1. 1. 1. 3. 1. 3. 3. 3. 1. 3. 1.
 3. 1. 1. 1. 3. 1. 3. 1. 1. 1. 1. 1. 1. 3. 3. 3. 1. 3. 1. 3. 1. 3. 3. 1.
 1. 3. 1. 1. 3. 1. 1. 1. 1. 3. 3. 1. 3. 1. 1. 3. 1. 3. 1. 3. 3. 1. 3. 1.
 3. 1. 1. 1. 1. 3. 3. 1. 3. 1. 3. 1. 3. 3. 1. 3. 1. 1. 3. 1. 1. 1. 1. 1.
 3. 3. 1. 1. 3. 3. 1. 3. 1. 3. 3. 1. 1. 3. 3. 3. 1. 3. 3. 3. 1. 1. 1. 1.
 3. 1. 1. 1. 3. 3. 1. 3. 1. 1. 3. 1. 3. 1. 1. 3. 3. 1. 3. 1. 1. 1. 1. 3.
 3. 3. 3. 1. 1. 1. 3. 1. 3. 3. 1. 1. 3. 3. 3. 1. 1. 3. 3. 1. 1. 1. 3. 3.
 3. 1. 3. 1. 1. 1. 3. 1. 1. 1. 3. 3. 3. 3. 1. 3. 1. 1. 1. 3. 3. 3. 3. 1.
 1. 1. 1. 1. 3. 3. 3. 1. 3. 1. 3. 1. 1. 3. 1. 1. 3. 1. 1. 3. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 3. 1. 1. 1. 3. 1. 1. 3. 3. 1. 3. 3. 3. 1. 1. 1. 3. 1. 1.
 1. 1. 3. 1. 1. 1. 3. 1. 1. 3. 1. 3. 3. 3. 1. 1. 1. 1. 3. 1. 1. 3. 3. 3.
 1. 3. 1. 1. 1. 1. 1. 3. 3. 1. 3. 3. 1. 1. 1. 1. 3. 1. 3. 3. 3. 1. 1. 3.
 1. 1. 3. 3. 1. 1. 3. 1. 1. 1. 3. 3. 1. 1. 3. 3. 3. 3. 1. 1. 1. 1. 1. 1.
 3. 3. 1. 3. 1. 1. 1. 1. 1. 1. 3. 3. 1. 3. 3

In [ ]:
def ensemble_combine(feats_pred, fieldpos_pred):
    times = 0
    count = 0
    home_wins = 0
    final_pred = []
    for i in range(0,len(fieldpos_test_data[1])):
        #print(fieldpos_test_data[1][i],feats_pred[i], fieldpos_pred[i], [(feats_pred[i][0] +   fieldpos_pred[i][0])/2,  (feats_pred[i][1] + fieldpos_pred[i][1])/2,  (feats_pred[i][2] + fieldpos_pred[i][2])/2])
        
        final_pred.append((feats_pred[i][0] +   fieldpos_pred[i][0])/2,  (feats_pred[i][1] + fieldpos_pred[i][1])/2,  (feats_pred[i][2] + fieldpos_pred[i][2])/2)
        #a = [(feats_pred[i][0] +  fieldpos_pred[i][0])/2,  (feats_pred[i][1] + fieldpos_pred[i][1])/2,  (feats_pred[i][2] + fieldpos_pred[i][2])/2]
        #a = [feats_pred[i][0],  feats_pred[i][1],  feats_pred[i][2] ]
        #a = [fieldpos_pred[i][0],  fieldpos_pred[i][1],  fieldpos_pred[i][2] ]

        m = max(a)
        it = a.index(m) + 1
        #predict vs actual
        if it == fieldpos_test_data[1][i]:
            times = times + 1
            if it == 1 or it == 3:
                home_wins +=1

        count = count + 1
#     print(times, count)
#     print(times/count)
#     print(home_wins/count)
